In [1]:
from manrs.models import Base, Report, Result
import config
from sqlalchemy.orm import sessionmaker
from sqlalchemy import or_, func
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [2]:
Session = sessionmaker(config.DB_ENGINE)
session = Session()
month = relativedelta(months=1)

In [3]:
asns = session.query(Result.asn).distinct().all()
print(f"Number of ASNs in database: {len(asns)}")

Number of ASNs in database: 190


In [5]:
# list all reports in database
reports = session.query(Report.id, Report.period_start).all()
print(f"Number of reports in database: {len(asns)}")

Number of reports in database: 190


In [6]:
# last report
last_report = session.query(func.max(Report.period_start)).one()[0]

In [7]:
def gen_dates(start):
    cursor = start
    while cursor < datetime.now() + relativedelta(years=1):
        cursor += month
        yield cursor

new_dates = list(gen_dates(last_report))

In [41]:
def sampler(mu, sigma, num):
    """ samples from a random distribution and clips values below 0"""
    samples = np.random.normal(mu, sigma, num)
    return np.clip(samples, a_min=0, a_max=None)

In [8]:
# add new reports to database
reports = []
for date in new_dates:
    end = new_dates[0] + month
    report = Report(period_start=date, period_end=end, date_finished=end, type="auto")
    session.add(report)

In [43]:
# a simple simulation assuming the data is a normal distribution
R = Result  # simple shortcut alias
for asn in [3356]: # asns
    rows = session.query(R.m1, R.m1c, R.m2, R.m2c, R.m3, R.m7irr).filter(Result.asn == asn).all()
    array = np.array(rows, dtype=float)
    mu = np.mean(array, axis=0)
    sigma = np.std(array, axis=0)
    
    m1    = sampler(mu[0], sigma[0], len(new_dates))
    m1c   = sampler(mu[1], sigma[1], len(new_dates))
    m2    = sampler(mu[2], sigma[2], len(new_dates))
    m2c   = sampler(mu[3], sigma[3], len(new_dates))
    m3    = sampler(mu[4], sigma[4], len(new_dates))
    m7irr = sampler(mu[5], sigma[5], len(new_dates))
    
    values = zip(new_dates, m1, m1c, m2, m2c, m3, m7irr)
   
